In [2]:
# connect to redis
import redis
import os
import base64
from PIL import Image
from io import BytesIO
import io, random
from datetime import datetime, timedelta
import json
from utils import img_to_txt


server = os.environ['redis_server'] if 'os.environ' in os.environ and len(os.environ['redis_server']) > 1 else 'localhost'
print(f"User Server {server}")
# connect with redis server as Bob
r = redis.Redis(host=server, port=6379)




User Server localhost


In [4]:



brust     = 1
count     = 0 





with r.pipeline() as pipe:
    for i in range(0, brust):
        random_id = int(random.random()*100000)

        image= {
                'request_time'   : str(datetime.today()),
                'image'          : img_to_txt("test_images/test.jpeg"),
                'status'         : 'pending'
                }
        
        # Publishing to the stream for testing
        pipe.publish('new_request', json.dumps(image))
        count+=1
        
    
    pipe.execute()
print(f"Request Done {count}")

Request Done 1


In [1]:
from collections import Counter
def millis_interval(end, start):
    """start and end are datetime instances"""
    diff = end - start
    millis = diff.days * 24 * 60 * 60 * 1000
    millis += diff.seconds * 1000
    millis += diff.microseconds / 1000
    return millis

count = 0
processing_time = []
task_done_time  = []
server_name     = []
not_processed   = 0



for k in r.scan_iter(match='*'):
    i = r.hgetall(k)
#     print(i)
    try:
        request_time    = datetime.strptime(i[b'request_time'].decode(), "%Y-%m-%d %H:%M:%S.%f")
        processed_time   = datetime.strptime(i[b'processed_time'].decode(), "%Y-%m-%d %H:%M:%S.%f")
        response_time   = datetime.strptime(i[b'received_time'].decode(), "%Y-%m-%d %H:%M:%S.%f")
        CSR_NAME        = i[b'server_name'].decode()
    
        #+ timedelta(hours=5)
        processing_time.append(millis_interval(processed_time, request_time ))
        task_done_time.append(millis_interval(processed_time, response_time  ))
        server_name.append(CSR_NAME)
    except Exception as e:
        print(e)
        pass
    
    count += 1
    
print(count)
print(f"Request Processed {brust - not_processed}")
print(f"Average Whole Request Processing Time this artitecture: {(sum(processing_time)/len(processing_time))/1000}")
print(f"Average Request Processed Time By CSR: {(sum(task_done_time)/len(task_done_time))/1000}")
print(f"Request Processed by {Counter(server_name)}")

NameError: name 'r' is not defined